In [1]:
import psycopg2
import os
import requests
import time
from dotenv import load_dotenv

In [2]:
load_dotenv()
TEST_PUUID = 'Wmig-c4JTkPXHkAJJ8NCEUE5FFpJyFfF0IpwXQfI3geaYExpuGTHUGON2EW4TnGOSqLY5LHxCL0xUQ'
TEST_MATCH_ID = 'NA1_5031446976'
RATE_LIMITS_PER_SECOND = 20
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
DEFAULT_REGION_CODE = 'euw1' 
DEFAULT_REGION = 'europe'

#default_region_code -> na1, euw1, kr
#default_region -> americas, europe, asia, sea

In [3]:
def create_connection():
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    return conn, cursor

In [4]:
def create_match_data_table():
    #create connection to db
    conn, cursor = create_connection()

    table_check_sql = """
    SELECT EXISTS (
        SELECT FROM pg_tables 
        WHERE schemaname = 'public' 
        AND tablename = 'match_data'
    );
    """

    create_table_sql = """
            CREATE TABLE match_data (
                id SERIAL PRIMARY KEY,
                match_id TEXT NOT NULL,
                region TEXT NOT NULL,
                game_mode TEXT NOT NULL,
                elo TEXT NOT NULL,
                version TEXT NOT NULL,
                blue_one TEXT NOT NULL,
                blue_two TEXT NOT NULL,
                blue_three TEXT NOT NULL,
                blue_four TEXT NOT NULL,
                blue_five TEXT NOT NULL,
                red_one TEXT NOT NULL,
                red_two TEXT NOT NULL,
                red_three TEXT NOT NULL,
                red_four TEXT NOT NULL,
                red_five TEXT NOT NULL,
                winning_team INTEGER NOT NULL CHECK (winning_team IN (0, 1)),
                UNIQUE (match_id)
            );
            """
    try:
        cursor.execute(table_check_sql)
        table_exists = cursor.fetchone()[0]
        if not table_exists:
            cursor.execute(create_table_sql)
            conn.commit()
            print("Table created successfully")
        else:
            print("Table exists")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    cursor.close()
    conn.close()

In [5]:
def get_user_matches(puuid = None, region=DEFAULT_REGION):
    if not puuid:
        raise Exception('Please Select puuid')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [6]:
def get_match_data(match_id = None, region=DEFAULT_REGION):
    if not match_id:
        raise Exception('Please Select Match ID')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [7]:
def get_elo(puuid, region=DEFAULT_REGION_CODE):
    if not puuid:
        raise Exception('Please Select Match ID')
    
    get_summoner_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }    
    try:
        summoner_response = requests.get(get_summoner_url,headers=headers)
        summoner_response.raise_for_status()
        summoner_response = summoner_response.json()['id']

        get_rank_url = f'https://{region}.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_response}'
        rank_response = requests.get(get_rank_url,headers=headers)
        rank_response.raise_for_status()
        return rank_response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')


In [8]:
def put_data(match_id, match_data):
    conn, cursor = create_connection()

    insert_sql = """
    INSERT INTO match_data (
        match_id, region, game_mode, elo, version,
        blue_one, blue_two, blue_three, blue_four, blue_five,
        red_one, red_two, red_three, red_four, red_five,
        winning_team
    ) VALUES (
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s,
        %s
    )
"""

    region = match_data['info']['platformId']
    game_mode = match_data["info"]["gameMode"]
    version = match_data["info"]["gameVersion"]
    team = [match_data['info']['participants'][i]['championName'] for i in range(10)]
    blue_one, blue_two, blue_three, blue_four, blue_five, red_one, red_two, red_three, red_four, red_five = team
    winning_team =  0 if match_data['info']['participants'][0]['win'] else 1

    elo_data = get_elo(match_data['metadata']['participants'][0])
    if elo_data:
        for entry in elo_data:
            if entry.get('queueType') == 'RANKED_SOLO_5x5':
                elo = f"{entry['tier']} {entry['rank']}"
                break
            elif entry.get('queueType') == 'RANKED_FLEX_5x5':
                elo = f"{entry['tier']} {entry['rank']}"
                break
        else:
            elo = "UNRANKED"
    else:
        elo = "UNRANKED"
    
    try:
        cursor.execute(insert_sql,(
            match_id, region, game_mode, elo, version,
            blue_one, blue_two, blue_three, blue_four, blue_five,
            red_one, red_two, red_three, red_four, red_five,
            winning_team
        ))
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)

    conn.close()
    cursor.close()    
    

In [9]:
def main(initial_puuid = None):
    #create match data if it does not exist
    create_match_data_table()
    unique_match_set = set()
    unique_users_set = {initial_puuid}
    unique_users_list = list(unique_users_set)
    current_index = 0

    while current_index <= 2500:
        cid = unique_users_list[current_index]
        current_index+=1

        if(current_index % 100 == 0):
            print(current_index)

        try:
            match_history = get_user_matches(cid)
            for match_id in match_history:
                if match_id not in unique_match_set:
                    try:
                        match_data = get_match_data(match_id)
                        if match_data['info']['gameMode'] in ('ARAM','CLASSIC'):
                            put_data(match_id, match_data)
                        players = match_data['metadata']['participants']
                        for player in players:
                            if player not in unique_users_set:
                                unique_users_set.add(player)
                                unique_users_list.append(player)
                        unique_match_set.add(match_id)
                        time.sleep(0.6)
                    except Exception as e:
                        print(f'Failed to retrieve data for match {match_id}: {e}')
        except Exception as e:
            print(f'Failed to retrieve match history for user {cid}: {e}')
        


main("rSUHzrv-Lm8lNJbGAVT1BiAdseDW1F5PxyzKn8wIZ27UQYIb27AXP6jLDZGXsgdq86Jjr2rs6KVFUw")

Table exists
Error Occured: 429 Client Error: Too Many Requests for url: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/QnKPis3YradC_aIX4_H0dsQ5oOZKoxhv3E4ob4ku2IotJxxGR5BwKrpPrGesL_n02EWJ-1WqTDsLIg
Error Occured: 429 Client Error: Too Many Requests for url: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/5ezg0f1kbOgYvZqSo7kWSZWpmMnU9oMk0wH5CnNu-W534j9MIBrADQ2rZJdEJ699gRZPcAmr50jssw
Error Occured: 429 Client Error: Too Many Requests for url: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/2y0M65x3RC3YZCBiJD2kN2ev6tCkGcSuJ1dRZi1suSU4D4Y60P7Q0g_z1fw4ETozVv36QIQTSwQDyA
Error Occured: 429 Client Error: Too Many Requests for url: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/7Oe0_ZbSJgmceOj-8a5FTckBn6kHHUjpW4A97WnfYHxKb9H1XJkfefmxrm5oZvBUDIDbJXRYDrImPQ
Error Occured: 429 Client Error: Too Many Requests for url: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/uTwbp2gnK732WhbdRmIsG5L0fEgdbNMt4uaTqfhrv

In [4]:
def verify_database():
    _, cursor = create_connection()

    try:
        cursor.execute("""SELECT count(*)
                       FROM match_data""")
        row_count = cursor.fetchone()[0]
        return row_count
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        return None

row_count = verify_database()
print(row_count)

72209


In [5]:
def process_elo(elo):
    return elo.split()[0]

def process_version(version):
    return '.'.join(version.split('.')[:2])

def unique_values():
    conn, cursor = create_connection()

    query_sql = """ 
        SELECT DISTINCT region AS value, 'region' AS column_name
        FROM match_data
        UNION ALL
        SELECT DISTINCT game_mode AS value, 'game_mode' AS column_name
        FROM match_data
        UNION ALL
        SELECT DISTINCT elo AS value, 'elo' AS column_name
        FROM match_data
        UNION ALL
        SELECT DISTINCT version AS value, 'version' AS column_name
        FROM match_data;
    """

    try:
        cursor.execute(query_sql)
        results = cursor.fetchall()

        unique_values_dict = {}
        for value, column_name in results:
            if column_name == 'elo':
                value = process_elo(value)
            elif column_name == 'version':
                value = process_version(value)
            
            if column_name not in unique_values_dict:
                unique_values_dict[column_name] = []
            if value not in unique_values_dict[column_name]:
                unique_values_dict[column_name].append(value)

        return unique_values_dict

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        conn.close()

    
print(unique_values())

{'region': ['EUW1', 'NA1'], 'game_mode': ['ARAM', 'CLASSIC'], 'version': ['14.6', '13.15', '14.5', '14.12', '13.18', '13.19', '13.16', '14.10', '13.20', '13.17', '13.11', '13.22', '13.21', '14.8', '14.13', '14.14', '14.2', '14.4', '14.11', '14.9', '13.7', '14.3', '14.1', '13.23', '13.10', '14.7', '13.8', '13.13', '13.14', '13.24', '13.6', '13.12', '13.9', '14.15'], 'elo': ['GOLD', 'PLATINUM', 'CHALLENGER', 'EMERALD', 'BRONZE', 'GRANDMASTER', 'SILVER', 'DIAMOND', 'IRON', 'MASTER', 'UNRANKED']}
